In [1]:
import pysftp
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import time
from time import sleep
import os
import shutil 
import sys
from tqdm import tqdm
now = datetime.datetime.now()
timestamp = str(now.strftime("%Y%m%d_%H%M%S"))

In [ ]:
#Downloading Updates
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys = None
# sftp = pysftp.Connection('sftp.talentreef.com', username=hidden, password=hidden, cnopts=cnopts)
# sftp.get('/outbound/pde/TCF_18703/TCF_Terminations/TCF_Terminations20210802-1300-23.csv')
# print("Process Complete")


In [ ]:
#Creating Employee Master Backup
print("Creating Employee Master Backup...") 
# for i in tqdm(range(10)):
#     sleep(0.1)
src = Path("../Resources/Source_data/master.csv")
dst = Path("../Resources/Source_data/Archive/MASTER_BACKUP.csv")
shutil.copy(src, dst)
print("Employee Master Backup Complete")

Creating Employee Master Backup...
Employee Master Backup Complete


In [15]:
#Validating Employee Update Data
##Checking for under age candidates
##Checking for missing required data SSN, BANK ROUTING, BANK ACCOUNT, EMAIL, PHONE
upval_employee_data_updates = pd.read_csv(Path("../Resources/Source_data/updates.csv"), header = 0, dtype = {"SSN" : str}) 
count_SSN = 0

upval_employee_data_updates["FLAG"] = np.where(upval_employee_data_updates["SSN"].isna(), "SSN Missing","")
count_SSN = upval_employee_data_updates["SSN"].isna().sum()
# count_SSN
upval_employee_data_updates 


##Checking for UNHIREABLE employee

,EMP_NO,FIRST_NAME,MIDDLE_INITIAL,LAST_NAME,SSN,ADDRESS,ADDRESS_2,CITY,STATE,ZIP,...,FEDFILINGSTATUS,WORKSTATE,BANK_RTN,BANK_ACC,BANK_NAME,BANK_TYPE,EMP_STATUS,PAYCODE,PAYRATE,FLAG
0,101010,Diamond,NaN,Zamora,123456799,142 w 22nd street,NaN,Houston,TX,77361,...,Single or Married filing separately,TX,91408734,16874136,Prinko Bank,Checking,Inactive,Hourly,18,
1,101022,Brittney,NaN,Hill,123456811,274 Roosevelt st,NaN,Sugar Land,TX,77078,...,Head of Household,TX,256074974,1235483141,Mega Bank,Checking,Active,Hourly,35,
2,101023,Ikeisha,NaN,Johnson,123456812,18164 hwy 65 ne 14 55011,14,Sugar Land,TX,77507,...,Single or Married filing separately,TX,124085244,1321541313,Prinko Bank,Checking,Active,Hourly,15,
3,101024,Kimberly,NaN,Flores Perez,123456813,6338 N Talman Ave,1,Woodlands,TX,77408,...,Single or Married filing separately,TX,124085244,1215445,Rolling Bank,Checking,Active,Hourly,30,
4,101025,Miracle,NaN,Thomas,NaN,8930 south Elizabeth,House,Woodlands,TX,77654,...,Head of Household,TX,91000019,21546931354,Mega Bank,Checking,Active,Hourly,40,SSN Missing
5,101026,Nallely,NaN,Ramirez,123456815,9233 S Tripp,NaN,Houston,TX,77215,...,Single or Married filing separately,TX,103100195,1564642132,Prinko Bank,Checking,Active,Hourly,50,
6,101027,Jermaine,T,Haynie,123456816,4406 Vernon Ave,NaN,Houston,TX,77555,...,Single or Married filing separately,TX,73905912,2136564123,Rolling Bank,Checking,Active,Hourly,45,
7,101028,Michaela,NaN,Rafferty,123456817,301 North Riverfront Drive 203,NaN,Richmond,TX,77412,...,Single or Married filing separately,TX,124085244,16848643139846,Mega Bank,Checking,Active,Hourly,17,
8,101029,Troydell,C,Hampton,123456818,187 East 24th Street,NaN,Houston,TX,77658,...,Head of Household,TX,41215663,164864318771,Prinko Bank,Checking,Active,Hourly,15,
9,101030,Teresa,T,Jones,123456819,414 w Lincoln hwy,NaN,Houston,TX,77258,...,Head of Household,TX,124085244,516461313,Rolling Bank,Checking,Active,Hourly,65,


In [29]:
#Updating Employee Master
upval_employee_data = pd.read_csv(Path("../Resources/Source_data/master.csv"), index_col = "EMP_NO", keep_default_na = False, dtype = {"SSN" : str, "BANK_RTN" : str, "BANK_ACC" : str})
upval_employee_data_updates = pd.read_csv(Path("../Resources/Source_data/updates.csv"), index_col = "EMP_NO", keep_default_na = False, dtype = {"SSN" : str, "BANK_RTN" : str, "BANK_ACC" : str}) 
# upval_employee_data = upval_employee_data.set_index(["EMP_NO"])
# upval_employee_data_updates = upval_employee_data_updates.set_index(["EMP_NO"])
upval_employee_data = upval_employee_data[~upval_employee_data.index.isin(upval_employee_data_updates.index)].reset_index()
# upval_employee_data = upval_employee_data.reset_index()
upval_employee_data_updates = upval_employee_data_updates.reset_index()
upval_employee_data = upval_employee_data.append(upval_employee_data_updates, ignore_index="True", verify_integrity="True").sort_values(by="EMP_NO").astype("str")
# upval_employee_data = upval_employee_data.replace("nan","")
# upval_employee_data = upval_employee_data[["EMP_NO","FIRST_NAME","MIDDLE_INITIAL","LAST_NAME","SSN","ADDRESS","ADDRESS_2","CITY","STATE","ZIP","PHONE_NO","EMAIL","JOBTITLE","HIREDATE","DOB","FEDFILINGSTATUS","WORKSTATE","BANK_RTN","BANK_ACC","BANK_NAME","BANK_TYPE","EMP_STATUS","PAYCODE","PAYRATE"]]
upval_employee_data.to_csv(Path("../Resources/Source_data/master1.csv"), index=False)

,EMP_NO,FIRST_NAME,MIDDLE_INITIAL,LAST_NAME,SSN,ADDRESS,ADDRESS_2,CITY,STATE,ZIP,...,DOB,FEDFILINGSTATUS,WORKSTATE,BANK_RTN,BANK_ACC,BANK_NAME,BANK_TYPE,EMP_STATUS,PAYCODE,PAYRATE
0,101000,Eric,,Lane,123456789,458 Country Club Dr.,,Sugar Land,TX,77078,...,06/05/1993,Single or Married filing separately,TX,256074974,1235483141,Mega Bank,Checking,Active,Salary,120000
1,101001,Jamel,,Bardney,123456790,2125 Georgia st,Apt#234,Sugar Land,TX,77507,...,07/09/2005,Single or Married filing separately,TX,124085244,1321541313,Prinko Bank,Checking,Active,Hourly,15
2,101002,Chudier,,Chuol,123456791,304 W 15th,,Woodlands,TX,77408,...,07/03/1993,Head of Household,TX,124085244,1215445,Rolling Bank,Checking,Active,Salary,75000
3,101003,Ivonne,,Peralta Velazquez,123456792,7420 Penn ave s,,Woodlands,TX,77654,...,12/15/1989,Married or filing jointly,TX,091000019,21546931354,Mega Bank,Checking,Active,Hourly,40
4,101004,Terano,,McCann,123456793,4233 Clark Drive,,Houston,TX,77215,...,03/26/2000,Single or Married filing separately,TX,103100195,1564642132,Prinko Bank,Checking,Active,Salary,75000
5,101005,Thomas,E,Hendricks,123456794,1115 E. 5th Street,,Houston,TX,77555,...,09/13/1971,Single or Married filing separately,TX,073905912,2136564123,Rolling Bank,Checking,Active,Hourly,45
6,101006,Marcell,,Mitchell,123456795,2556 Pinkney St,,Houston,TX,77412,...,05/26/2004,Single or Married filing separately,TX,124085244,16848643139846,Mega Bank,Checking,Active,Hourly,17
7,101007,Valarie,,Lopez,123456796,626 Roush Lane,,Houston,TX,77658,...,10/08/1995,Head of Household,TX,041215663,164864318771,Prinko Bank,Checking,Active,Hourly,15
8,101008,James,E,Wimbush,123456797,240 Hobart St.,,Houston,TX,77258,...,11/03/1974,Single or Married filing separately,TX,124085244,516461313,Rolling Bank,Checking,Active,Salary,95000
9,101009,Dewayne,,Chester,123456798,4615 Johnson St.,,Houston,TX,77685,...,03/01/1974,Single or Married filing separately,TX,124085244,569746414,Great Bank,Checking,Active,Hourly,15


In [208]:
#Loading Employee Data
employee_data = pd.read_csv(Path("../Resources/Source_data/master1.csv"), header = 0, usecols = ["EMP_NO","EMP_STATUS","PAYCODE","PAYRATE"])
hour_data = pd.read_csv(Path("../Resources/Source_data/hours.csv")) 
employee_ach_data = pd.read_csv(Path("../Resources/Source_data/master1.csv"), header = 0, usecols = ["EMP_NO","FIRST_NAME","LAST_NAME","BANK_RTN","BANK_ACC"], dtype = {"BANK_RTN" : str, "BANK_ACC" : str})

In [209]:
#Calculating Hourly Wages
hourly_emp_data = employee_data.loc[employee_data["PAYCODE"] == "Hourly"]
hourly_emp_data = employee_data.loc[employee_data["EMP_STATUS"] == "Active"]
hourly_grid = pd.merge(hourly_emp_data, hour_data, how = "inner", on = "EMP_NO")
hourly_grid["WAGES"] = np.where(hourly_grid["HOURS"] > 40, 
    ((hourly_grid["HOURS"]-40)*1.5*hourly_grid["PAYRATE"])+ (hourly_grid["PAYRATE"]*40), 
    (hourly_grid["HOURS"] * hourly_grid["PAYRATE"]))
hourly_grid.drop(["EMP_STATUS","PAYCODE","PAYRATE","HOURS"], axis=1).round(2)

,EMP_NO,WAGES
0,101001,847.5
1,101003,1400.0
2,101005,1800.0
3,101006,1062.5
4,101007,825.0
5,101009,600.0
6,101010,680.0
7,101011,1400.0


In [210]:
#Calculating Salary Wages
salary_grid = employee_data.loc[employee_data["PAYCODE"] == "Salary"]
salary_grid = employee_data.loc[employee_data["EMP_STATUS"] == "Active"]
salary_grid["WAGES"] = salary_grid["PAYRATE"] / 52
salary_grid.drop(["EMP_STATUS","PAYCODE","PAYRATE"], axis=1).round(2)

,EMP_NO,WAGES
0,101000,2307.69
1,101001,0.29
2,101002,1442.31
3,101003,0.77
4,101004,1442.31
5,101005,0.87
6,101006,0.33
7,101007,0.29
8,101008,1826.92
9,101009,0.29


In [211]:
#Combining Salary and Hourly Wages
all_wages = hourly_grid.append(salary_grid, ignore_index=True, verify_integrity= True)
all_wages.drop(["EMP_STATUS","PAYRATE","HOURS"], axis=1).round(2)

,EMP_NO,PAYCODE,WAGES
0,101001,Hourly,847.50
1,101003,Hourly,1400.00
2,101005,Hourly,1800.00
3,101006,Hourly,1062.50
4,101007,Hourly,825.00
5,101009,Hourly,600.00
6,101010,Hourly,680.00
7,101011,Hourly,1400.00
8,101000,Salary,2307.69
9,101001,Hourly,0.29


In [23]:
#Creating ACH File
employee_ach_data["NAME"] =  employee_ach_data["FIRST_NAME"].str.upper() + [" "] + employee_ach_data["LAST_NAME"].str.upper()
employee_ach_data = pd.merge(all_wages, employee_ach_data, how="inner", on = "EMP_NO")
employee_ach_data = employee_ach_data.drop(["FIRST_NAME","LAST_NAME","EMP_STATUS","PAYCODE", "PAYRATE", "HOURS"], axis=1).round(2)
employee_ach_data

NameError: name 'employee_ach_data' is not defined